In [ ]:
import numpy as np
import numpy.linalg as la
import copy
# Define the matrix A
A = np.array([[1, 2, 3],
              [0, 1, 4]])

In [6]:
def gram_schmidt(A):
    """Compute an orthonormal matrix using the Gram-Schmidt process on the rows of A"""
    A = A.copy()
    for k in range(A.shape[0]):
        v_k = A[k].copy()
        for j in range(k):
            u_j = A[j]
            proj = np.dot(v_k, u_j) / np.dot(u_j, u_j) * u_j
            v_k -= proj
        norm = np.linalg.norm(v_k)
        if norm > 1e-10:
            A[k] = v_k / norm
        else:
            A[k] = v_k  # or raise an error if you expect full rank
    return A

In [7]:
import numpy as np

def test_qr_decomp(qr_func, tol=1e-8, complex_test=True):
    def check_orthonormal(Q):
        I = np.eye(Q.shape[1], dtype=Q.dtype)
        err = np.linalg.norm(Q.T.conj() @ Q - I)
        return err < tol, err

    def check_upper_triangular(R):
        err = np.linalg.norm(np.tril(R, -1))
        return err < tol, err

    def check_reconstruction(A, Q, R):
        err = np.linalg.norm(Q @ R - A)
        return err < tol * max(1, np.linalg.norm(A)), err

    def check_no_nan_inf(X):
        return not (np.isnan(X).any() or np.isinf(X).any())

    tests = []
    passed = 0
    failed = 0

    def run_test(name, A):
        nonlocal passed, failed
        print(f"\nTest: {name}")
        try:
            Q, R = qr_func(A)
        except Exception as e:
            print(f"  FAILED: Exception during QR: {e}")
            failed += 1
            return

        # Shape checks
        if Q.shape[0] != A.shape[0] or R.shape[1] != A.shape[1]:
            print(f"  FAILED: Shape mismatch. Q: {Q.shape}, R: {R.shape}, A: {A.shape}")
            failed += 1
            return

        # Orthonormality
        ortho, ortho_err = check_orthonormal(Q)
        print(f"  Orthonormality: {'PASS' if ortho else 'FAIL'} (err={ortho_err:.2e})")

        # Upper triangular
        upper, upper_err = check_upper_triangular(R)
        print(f"  Upper Triangular: {'PASS' if upper else 'FAIL'} (err={upper_err:.2e})")

        # Reconstruction
        recon, recon_err = check_reconstruction(A, Q, R)
        print(f"  Reconstruction: {'PASS' if recon else 'FAIL'} (err={recon_err:.2e})")

        # NaN/Inf
        naninf_Q = check_no_nan_inf(Q)
        naninf_R = check_no_nan_inf(R)
        print(f"  Q NaN/Inf: {'PASS' if naninf_Q else 'FAIL'}")
        print(f"  R NaN/Inf: {'PASS' if naninf_R else 'FAIL'}")

        # Final pass/fail
        if ortho and upper and recon and naninf_Q and naninf_R:
            print("  Test PASSED")
            passed += 1
        else:
            print("  Test FAILED")
            failed += 1

    # Random matrices
    np.random.seed(42)
    for m, n in [(3, 3), (5, 3), (3, 5), (10, 10), (8, 2), (2, 8)]:
        A = np.random.randn(m, n)
        run_test(f"Random real {m}x{n}", A)

    # Rank-deficient
    A = np.random.randn(5, 5)
    A[:, 2] = A[:, 1]  # duplicate column
    run_test("Rank-deficient (duplicate columns)", A)
    A = np.zeros((4, 4))
    run_test("Zero matrix", A)

    # Known cases
    run_test("Identity", np.eye(4))
    run_test("Diagonal", np.diag([1, 2, 3, 4]))
    run_test("Upper triangular", np.triu(np.ones((4, 4))))
    Q_known = np.array([[1, 0], [0, -1]])
    run_test("Known orthogonal", Q_known)

    # Edge sizes
    run_test("1x1", np.array([[7.0]]))
    run_test("1xn", np.random.randn(1, 5))
    run_test("nx1", np.random.randn(5, 1))
    run_test("Empty 0x0", np.empty((0, 0)))
    run_test("Empty 0x3", np.empty((0, 3)))
    run_test("Empty 3x0", np.empty((3, 0)))

    # Complex matrices
    if complex_test:
        A = np.random.randn(4, 4) + 1j * np.random.randn(4, 4)
        run_test("Random complex 4x4", A)
        A = np.eye(3, dtype=complex)
        run_test("Complex identity", A)

    print(f"\nSummary: {passed} PASSED, {failed} FAILED, {passed+failed} total tests.")

# Example usage with numpy's QR
if __name__ == "__main__":
    test_qr_decomp(lambda A: np.linalg.qr(A, mode='reduced'))


Test: Random real 3x3
  Orthonormality: PASS (err=2.29e-16)
  Upper Triangular: PASS (err=0.00e+00)
  Reconstruction: PASS (err=4.38e-16)
  Q NaN/Inf: PASS
  R NaN/Inf: PASS
  Test PASSED

Test: Random real 5x3
  Orthonormality: PASS (err=7.87e-16)
  Upper Triangular: PASS (err=0.00e+00)
  Reconstruction: PASS (err=1.42e-15)
  Q NaN/Inf: PASS
  R NaN/Inf: PASS
  Test PASSED

Test: Random real 3x5
  Orthonormality: PASS (err=1.80e-16)
  Upper Triangular: PASS (err=0.00e+00)
  Reconstruction: PASS (err=1.13e-15)
  Q NaN/Inf: PASS
  R NaN/Inf: PASS
  Test PASSED

Test: Random real 10x10
  Orthonormality: PASS (err=1.56e-15)
  Upper Triangular: PASS (err=0.00e+00)
  Reconstruction: PASS (err=3.40e-15)
  Q NaN/Inf: PASS
  R NaN/Inf: PASS
  Test PASSED

Test: Random real 8x2
  Orthonormality: PASS (err=1.26e-16)
  Upper Triangular: PASS (err=0.00e+00)
  Reconstruction: PASS (err=7.23e-16)
  Q NaN/Inf: PASS
  R NaN/Inf: PASS
  Test PASSED

Test: Random real 2x8
  Orthonormality: PASS (err=5.